In [21]:
import importlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.average_true_range as atr
import apollo.calculations.hull_moving_average as hma
import apollo.calculations.keltner_channel as kc

importlib.reload(yac)
importlib.reload(hma)
importlib.reload(kc)
importlib.reload(atr);

In [23]:
ticker = "SPY"
start_date = "2023-01-01"
end_date = "2024-01-01"

api_connector = yac.YahooApiConnector(ticker, start_date, end_date)
dataframe = api_connector.request_or_read_prices()

dataframe;

In [24]:
WINDOW_SIZE = 20

hma_calculator = hma.HullMovingAverageCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

hma_calculator.calculate_hull_moving_average()

atr_calculator = atr.AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

atr_calculator.calculate_average_true_range()

kc_calculator = kc.KeltnerChannelCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
    volatility_multiplier=1.1,
)

kc_calculator.calculate_keltner_channel()

dataframe.dropna(inplace=True)

dataframe

,ticker,open,high,low,close,adj close,volume,hma,tr,atr,lkc_bound,ukc_bound
date,,,,,,,,,,,,
2023-02-28,SPY,397.230011,399.279999,396.149994,396.260010,389.118561,96438600,390.559450,3.130005,5.858705,384.114874,397.004025
2023-03-01,SPY,395.410004,396.690002,393.380005,394.739990,387.625946,99706800,388.575794,3.309998,5.731270,382.271398,394.880191
2023-03-02,SPY,392.679993,398.690002,392.329987,397.809998,390.640594,85127800,387.259368,6.360016,5.762707,380.920391,393.598346
2023-03-03,SPY,399.709991,404.450012,399.029999,404.190002,396.905609,90120000,387.268756,6.640015,5.806572,380.881526,393.655985
2023-03-06,SPY,405.049988,407.450012,404.010010,404.470001,397.180573,72795900,388.297752,3.440002,5.688244,382.040683,394.554820
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,SPY,473.859985,475.380005,471.700012,473.649994,472.182892,67126600,473.633468,3.679993,4.331225,468.869121,478.397816
2023-12-26,SPY,474.070007,476.579987,473.989990,475.649994,474.176697,55387000,474.636392,2.929993,4.261163,469.949113,479.323672
2023-12-27,SPY,475.440002,476.660004,474.890015,476.510010,475.034058,68000300,475.594958,1.769989,4.136605,471.044693,480.145224


In [25]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["adj close"],
        line={"color": "blue", "width": 1},
        name="Adj close",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["lkc_bound"],
        line={"color": "red", "width": 1},
        name="MNMA",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["ukc_bound"],
        line={"color": "red", "width": 1},
        name="Upper Bound",
    ),
    row=1,
    col=1,
)

fig.update_layout(
    title={"text": f"{ticker}", "x": 0.5},
    height=650,
    plot_bgcolor="#EFF5F8",
)

fig.update_yaxes(visible=False, secondary_y=True)